In [339]:
import uproot, numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as linalg
from scipy import stats
import random
from numpy import linalg as LA
from random import sample
from sklearn.decomposition import PCA
import pandas
from scipy import signal
from sklearn.linear_model import LinearRegression
import shapely
from termcolor import colored
from shapely.geometry import LineString, Point
class Events:
    def __init__(self, filename, treename):
        file = uproot.open(filename)
        tree = file[treename]
        self.u_x = tree["u_x"].array(library="np")
        self.u_z = tree["u_z"].array(library="np")
        self.u_energy = tree["u_energies"].array(library="np")
        self.u_pi = tree["u_pi"].array(library="np")
        self.vtx_u_z = tree["vtx_u_z"].array(library="np")
        self.end_u_z = tree["end_u_z"].array(library="np")
        self.v_x = tree["v_x"].array(library="np")
        self.v_z = tree["v_z"].array(library="np")
        self.v_energy = tree["v_energies"].array(library="np")
        self.v_pi = tree["v_pi"].array(library="np")
        self.vtx_v_z = tree["vtx_v_z"].array(library="np")
        self.end_v_z = tree["end_v_z"].array(library="np")
        self.w_x = tree["w_x"].array(library="np")
        self.w_z = tree["w_z"].array(library="np")
        self.w_energy = tree["w_energies"].array(library="np")
        self.w_pi = tree["w_pi"].array(library="np")
        self.vtx_w_z = tree["vtx_w_z"].array(library="np")
        self.end_w_z = tree["end_w_z"].array(library="np")
        self.event=tree["event"].array(library="np")
        self.vtx_x = tree["vtx_x"].array(library="np")
        self.end_x = tree["end_x"].array(library="np")
        file.close()


In [340]:
events = Events("hierarchywriting_nue_306.root", "hierarchy_writing")

In [341]:
def event_display(n):
    pion_events = events.w_pi[n]
    pion_indices = np.where(pion_events==1)
    not_pion_indices = np.where(pion_events!=1)
    energy = events.u_energy[n]
    pion_x = events.w_x[n][pion_indices]
    pion_z = events.w_z[n][pion_indices]
    x = events.w_x[n][not_pion_indices]
    y = events.w_z[n][not_pion_indices]
    pion_adc = events.w_energy[n][pion_indices]

In [342]:
def track_split_300(n, number):
    pfo_x = events.w_x[n]
    pfo_z = events.w_z[n]
    pca1=PCA(2)
    total_hits = np.concatenate((pfo_x[:, None], pfo_z[:, None]), axis=1)
    pca1.fit(total_hits)
    pca1.mean_ = [events.vtx_x[n][0], events.vtx_w_z[n][0]]
    Z = pca1.transform(total_hits)
    end_x = events.end_x[n]
    end_z = events.end_w_z[n] 
    
    max_value = max(Z[:,0])
    min_value = min(Z[:,0])
    rad_length = 14
    half_rad_length = rad_length/2
    length = (max_value - min_value)
    number_bins = np.int(np.ceil(length/half_rad_length))
    maximum = number_bins*half_rad_length + min_value
    bin_edges = np.linspace(min_value, maximum, number_bins +1)
    bin_edges_fraction = (bin_edges/rad_length) 
    bin_indices = np.digitize(Z[:, 0], bin_edges) -1 
    energy_array = events.w_energy[n]
    energy_profile = np.zeros(number_bins)

    for i in range(len(energy_array)):
        index = bin_indices[i] 
        energy_profile[index]=energy_array[i]+energy_profile[index]
    energy_profile = energy_profile[::-1]
    
    fractional_energy = energy_profile/sum(energy_profile)
    
    gradient = np.gradient(energy_profile)
    
    bin_number = np.int(np.ceil(len(gradient)/number))
     
    mean_grad = np.convolve(gradient, np.ones(number)/number, mode='valid')
    mean_energy = np.convolve(fractional_energy, np.ones(number)/number, mode='valid')
    
    track_transition=[]

    for p in range(len(mean_grad) - 2):
        if mean_grad[p] < mean_grad[p+1] < mean_grad[p+2] and mean_grad[p+2] - mean_grad[p+1] > 300:
            print(p) 
            track_transition.append(bin_edges_fraction[:-number][p+1])
            
    if not track_transition:
        pass
    else:
        track_cut = track_transition[0]
        bin_transition = track_cut * rad_length
        shifted_track_cut = track_cut + abs(min(bin_edges_fraction))
        
    if not track_transition:
        pass
    else:
        split_metric = []
        end_PCA = pca1.transform([[end_x[0], end_z[0]]]) - min_value
        shifted_Z = Z - min_value
        track_start_x=(pca1.inverse_transform(bin_transition)[:, 0][0])
        split_goodness = (abs(bin_transition) - abs(end_PCA))
        split_metric.append(split_goodness)
        return split_metric[0][0][0]